In [33]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler, MinMaxScaler
from xverse.transformer import WOE
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.impute import SimpleImputer

In [6]:
def load_data(file_path):
    return pd.read_csv(file_path)

# Load data
file_path = '../data/data2.csv'
data = load_data(file_path)

In [7]:
# Define reference date for recency calculation
reference_date = pd.to_datetime('2024-01-01')

# Create Aggregate Features
def create_aggregate_features(data):
    data['total_transaction_amount'] = data.groupby('CustomerId')['Amount'].transform('sum')
    data['average_transaction_amount'] = data.groupby('CustomerId')['Amount'].transform('mean')
    data['transaction_count'] = data.groupby('CustomerId')['Amount'].transform('count')
    data['std_transaction_amount'] = data.groupby('CustomerId')['Amount'].transform('std')
    return data
data=create_aggregate_features(data)
data.head()

,TransactionId,BatchId,AccountId,SubscriptionId,CustomerId,CurrencyCode,CountryCode,ProviderId,ProductId,ProductCategory,ChannelId,Amount,Value,TransactionStartTime,PricingStrategy,FraudResult,total_transaction_amount,average_transaction_amount,transaction_count,std_transaction_amount
0,76871,36123,3957,887,4406,UGX,256,6,10,airtime,3,1000.0,1000,2018-11-15T02:18:49Z,2,0,109921.75,923.712185,119,3042.294251
1,73770,15642,4841,3829,4406,UGX,256,4,6,financial_services,2,-20.0,20,2018-11-15T02:19:08Z,2,0,109921.75,923.712185,119,3042.294251
2,26203,53941,4229,222,4683,UGX,256,6,1,airtime,3,500.0,500,2018-11-15T02:44:21Z,2,0,1000.00,500.000000,2,0.000000
3,380,102363,648,2185,988,UGX,256,1,21,utility_bill,3,20000.0,21800,2018-11-15T03:32:55Z,2,0,228727.20,6019.136842,38,17169.241610
4,28195,38780,4841,3829,988,UGX,256,4,6,financial_services,2,-644.0,644,2018-11-15T03:34:21Z,2,0,228727.20,6019.136842,38,17169.241610


Extract Features

In [8]:
def extract_time_features(data):
    data['TransactionStartTime'] = pd.to_datetime(data['TransactionStartTime'])
    data['TransactionHour'] = data['TransactionStartTime'].dt.hour
    data['TransactionDay'] = data['TransactionStartTime'].dt.day
    data['TransactionMonth'] = data['TransactionStartTime'].dt.month
    data['TransactionYear'] = data['TransactionStartTime'].dt.year
    return data

# Feature Extraction
data = extract_time_features(data)
print("Data with Time Features:")
print(data[['TransactionStartTime', 'TransactionHour', 'TransactionDay', 'TransactionMonth', 'TransactionYear']].head())

Data with Time Features:
       TransactionStartTime  TransactionHour  TransactionDay  \
0 2018-11-15 02:18:49+00:00                2              15   
1 2018-11-15 02:19:08+00:00                2              15   
2 2018-11-15 02:44:21+00:00                2              15   
3 2018-11-15 03:32:55+00:00                3              15   
4 2018-11-15 03:34:21+00:00                3              15   

   TransactionMonth  TransactionYear  
0                11             2018  
1                11             2018  
2                11             2018  
3                11             2018  
4                11             2018  


In [9]:
# Encode Categorical Variables: One-Hot Encoding
def one_hot_encode(data, categorical_features):
    data = pd.get_dummies(data, columns=categorical_features)
    return data

categorical_features = data.select_dtypes(include=['object']).columns.tolist()
data_one_hot_encoded = one_hot_encode(data, categorical_features)
print("One-Hot Encoded Data:")
print(data_one_hot_encoded.head())

One-Hot Encoded Data:
   TransactionId  BatchId  AccountId  SubscriptionId  CustomerId  CountryCode  \
0          76871    36123       3957             887        4406          256   
1          73770    15642       4841            3829        4406          256   
2          26203    53941       4229             222        4683          256   
3            380   102363        648            2185         988          256   
4          28195    38780       4841            3829         988          256   

   ProviderId  ProductId  ChannelId   Amount  ...  CurrencyCode_UGX  \
0           6         10          3   1000.0  ...              True   
1           4          6          2    -20.0  ...              True   
2           6          1          3    500.0  ...              True   
3           1         21          3  20000.0  ...              True   
4           4          6          2   -644.0  ...              True   

  ProductCategory_airtime  ProductCategory_data_bundles  \
0    

In [10]:
# Encode Categorical Variables: Label Encoding
def label_encode(data, categorical_features):
    le = LabelEncoder()
    for feature in categorical_features:
        data[feature] = le.fit_transform(data[feature].astype(str))
    return data

data_label_encoded = label_encode(data, categorical_features)
print("Label Encoded Data:")
print(data_label_encoded.head())

Label Encoded Data:
   TransactionId  BatchId  AccountId  SubscriptionId  CustomerId  \
0          76871    36123       3957             887        4406   
1          73770    15642       4841            3829        4406   
2          26203    53941       4229             222        4683   
3            380   102363        648            2185         988   
4          28195    38780       4841            3829         988   

   CurrencyCode  CountryCode  ProviderId  ProductId  ProductCategory  ...  \
0             0          256           6         10                0  ...   
1             0          256           4          6                2  ...   
2             0          256           6          1                0  ...   
3             0          256           1         21                8  ...   
4             0          256           4          6                2  ...   

   PricingStrategy  FraudResult  total_transaction_amount  \
0                2            0                

In [11]:
# Handle Missing Values
def handle_missing_values(data, strategy='mean'):
    numerical_features = data.select_dtypes(include=[np.number]).columns.tolist()
    imputer = SimpleImputer(strategy=strategy)
    data[numerical_features] = imputer.fit_transform(data[numerical_features])
    return data

# Handle Missing Values
data = handle_missing_values(data, strategy='mean')

In [12]:
data.isna().sum()

TransactionId                 0
BatchId                       0
AccountId                     0
SubscriptionId                0
CustomerId                    0
CurrencyCode                  0
CountryCode                   0
ProviderId                    0
ProductId                     0
ProductCategory               0
ChannelId                     0
Amount                        0
Value                         0
TransactionStartTime          0
PricingStrategy               0
FraudResult                   0
total_transaction_amount      0
average_transaction_amount    0
transaction_count             0
std_transaction_amount        0
TransactionHour               0
TransactionDay                0
TransactionMonth              0
TransactionYear               0
dtype: int64

In [13]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 95662 entries, 0 to 95661
Data columns (total 24 columns):
 #   Column                      Non-Null Count  Dtype              
---  ------                      --------------  -----              
 0   TransactionId               95662 non-null  float64            
 1   BatchId                     95662 non-null  float64            
 2   AccountId                   95662 non-null  float64            
 3   SubscriptionId              95662 non-null  float64            
 4   CustomerId                  95662 non-null  float64            
 5   CurrencyCode                95662 non-null  float64            
 6   CountryCode                 95662 non-null  float64            
 7   ProviderId                  95662 non-null  float64            
 8   ProductId                   95662 non-null  float64            
 9   ProductCategory             95662 non-null  float64            
 10  ChannelId                   95662 non-null  float64       

In [14]:
def normalize_features(data):
    numerical_features = data.select_dtypes(include=[np.number]).columns.tolist()
    scaler = MinMaxScaler()
    data[numerical_features] = scaler.fit_transform(data[numerical_features])
    return data

# Normalize Features
normalized_data = normalize_features(data.copy())
print("Normalized Data:")
print(normalized_data.head())

Normalized Data:
   TransactionId   BatchId  AccountId  SubscriptionId  CustomerId  \
0       0.546417  0.258949   0.817355        0.183078    0.589061   
1       0.524374  0.112122   1.000000        0.791684    0.589061   
2       0.186252  0.386684   0.873554        0.045511    0.626103   
3       0.002694  0.733818   0.133678        0.451593    0.131987   
4       0.200412  0.277996   1.000000        0.791684    0.131987   

   CurrencyCode  CountryCode  ProviderId  ProductId  ProductCategory  ...  \
0           0.0          0.0         1.0   0.346154             0.00  ...   
1           0.0          0.0         0.6   0.192308             0.25  ...   
2           0.0          0.0         1.0   0.000000             0.00  ...   
3           0.0          0.0         0.0   0.769231             1.00  ...   
4           0.0          0.0         0.6   0.192308             0.25  ...   

   PricingStrategy  FraudResult  total_transaction_amount  \
0              0.5          0.0             

In [15]:
# Calculate RFM
def calculate_rfm(data):
    rfm = data.groupby('CustomerId').agg({
        'TransactionStartTime': lambda x: (pd.Timestamp.now(tz='UTC')- x.max()).days,  # Recency
        'CustomerId': 'count',                                         # Frequency
        'Amount': ['sum', 'mean']                           # Monetary and Spend
    }).reset_index()
    rfm.columns = ['CustomerId', 'recency', 'frequency', 'monetary', 'spend']
    rfm.fillna(0, inplace=True)
    return rfm
rfm=calculate_rfm(data)
rfm

,CustomerId,recency,frequency,monetary,spend
0,1.0,2026,1,-10000.0,-10000.00000
1,2.0,2026,1,-10000.0,-10000.00000
2,3.0,2026,1,-10000.0,-10000.00000
3,4.0,2026,1,-10000.0,-10000.00000
4,5.0,2026,1,-10000.0,-10000.00000
...,...,...,...,...,...
3737,7475.0,1980,3,-6000.0,-2000.00000
3738,7476.0,1976,23,-213400.0,-9278.26087
3739,7477.0,1998,1,-2000.0,-2000.00000
3740,7478.0,1954,1,500.0,500.00000


In [44]:
# Calculate RFMS Score
def calculate_rfms_score(rfm):
    rfm['rfms_score'] = (0.25 * rfm['recency']) + (0.25 * rfm['frequency']) + (0.25 * rfm['monetary']) + (0.25 * rfm['spend'])
    return rfm

rfm = calculate_rfms_score(rfm)
rfm


,CustomerId,recency,frequency,monetary,spend,rfms_score,user_class,Label
0,1.0,2026,1,-10000.0,-10000.00000,-4493.250000,bad,Bad
1,2.0,2026,1,-10000.0,-10000.00000,-4493.250000,bad,Bad
2,3.0,2026,1,-10000.0,-10000.00000,-4493.250000,bad,Bad
3,4.0,2026,1,-10000.0,-10000.00000,-4493.250000,bad,Bad
4,5.0,2026,1,-10000.0,-10000.00000,-4493.250000,bad,Bad
...,...,...,...,...,...,...,...,...
3737,7475.0,1980,3,-6000.0,-2000.00000,-1504.250000,bad,Bad
3738,7476.0,1976,23,-213400.0,-9278.26087,-55169.815217,bad,Good
3739,7477.0,1998,1,-2000.0,-2000.00000,-500.250000,bad,Bad
3740,7478.0,1954,1,500.0,500.00000,738.750000,bad,Bad


In [48]:
# Classify Users
def classify_users(rfm, threshold):
    rfm['user_class'] = rfm['rfms_score'].apply(lambda x: 'good' if x >= threshold else 'bad')
    return rfm
# Define threshold for good/bad classification
threshold = 5000
rfm = classify_users(rfm, threshold)
print(rfm['user_class'] .value_counts())
rfm

user_class
good    2307
bad     1435
Name: count, dtype: int64


,CustomerId,recency,frequency,monetary,spend,rfms_score,user_class,Label
0,1.0,2026,1,-10000.0,-10000.00000,-4493.250000,bad,Bad
1,2.0,2026,1,-10000.0,-10000.00000,-4493.250000,bad,Bad
2,3.0,2026,1,-10000.0,-10000.00000,-4493.250000,bad,Bad
3,4.0,2026,1,-10000.0,-10000.00000,-4493.250000,bad,Bad
4,5.0,2026,1,-10000.0,-10000.00000,-4493.250000,bad,Bad
...,...,...,...,...,...,...,...,...
3737,7475.0,1980,3,-6000.0,-2000.00000,-1504.250000,bad,Bad
3738,7476.0,1976,23,-213400.0,-9278.26087,-55169.815217,bad,Good
3739,7477.0,1998,1,-2000.0,-2000.00000,-500.250000,bad,Bad
3740,7478.0,1954,1,500.0,500.00000,738.750000,bad,Bad


trying different classification

In [20]:
# Define boundary values
high_freq_boundary = rfm['frequency'].quantile(0.75)
high_monetary_boundary = rfm['monetary'].quantile(0.75)
low_recency_boundary = rfm['recency'].quantile(0.25)
# Assign labels based on boundary
rfm['Label'] = 'Good'
rfm.loc[(rfm['frequency'] <= high_freq_boundary) & 
                (rfm['monetary'] <= high_monetary_boundary) & 
                (rfm['recency'] >= low_recency_boundary), 'Label'] = 'Bad'
print(rfm['Label'] .value_counts())
# Display the first few rows of RFMS scores with labels
rfm

Label
Bad     2249
Good    1493
Name: count, dtype: int64


,CustomerId,recency,frequency,monetary,spend,rfms_score,user_class,Label
0,1.0,2026,1,-10000.0,-10000.00000,-9492.750000,bad,Bad
1,2.0,2026,1,-10000.0,-10000.00000,-9492.750000,bad,Bad
2,3.0,2026,1,-10000.0,-10000.00000,-9492.750000,bad,Bad
3,4.0,2026,1,-10000.0,-10000.00000,-9492.750000,bad,Bad
4,5.0,2026,1,-10000.0,-10000.00000,-9492.750000,bad,Bad
...,...,...,...,...,...,...,...,...
3737,7475.0,1980,3,-6000.0,-2000.00000,-4502.750000,bad,Bad
3738,7476.0,1976,23,-213400.0,-9278.26087,-161858.315217,bad,Good
3739,7477.0,1998,1,-2000.0,-2000.00000,-1499.750000,bad,Bad
3740,7478.0,1954,1,500.0,500.00000,989.250000,good,Bad


In [27]:
import matplotlib
matplotlib.use('TkAgg')  # Choose an appropriate backend

# Plot RFMS space
def plot_rfms_space(rfm):
    sns.scatterplot(data=rfm, x='recency', y='frequency', hue='Label')
    plt.title('RFMS Space')
    plt.show()
plot_rfms_space(rfm)

In [49]:
def calculate_woe_continuous(data, feature, target):
    data['bins'], bins = pd.qcut(data[feature], 10, retbins=True, duplicates='drop')
    
    grouped = data.groupby(['bins', target], observed=True).size().unstack().fillna(0)
    
    # Check if grouped DataFrame is empty
    if grouped.empty:
        default_values = pd.DataFrame({
            'event_perc': [0.5],  # Default value for event percentage
            'non_event_perc': [0.5],  # Default value for non-event percentage
            'woe': [0.0]  # Default value for WOE
        })
        return default_values['woe']
    
    # Calculate percentage of events and non-events in each bin
    grouped['event_perc'] = grouped[1] / (grouped[0] + grouped[1] + 0.001)  # Add 0.001 to avoid division by zero
    grouped['non_event_perc'] = grouped[0] / (grouped[0] + grouped[1] + 0.001)
    
    # Calculate WOE
    grouped['woe'] = np.log(grouped['non_event_perc'] / grouped['event_perc'] + 0.001)  # Add 0.001 to avoid log(0)
    
    return grouped['woe']


In [40]:
def calculate_woe_categorical(data, feature, target):
    grouped = data.groupby([feature, target], observed=True).size().unstack().fillna(0)
    
    # Calculate percentage of events and non-events for each category
    grouped['event_perc'] = grouped[1] / (grouped[0] + grouped[1] + 0.001)  # Add 0.001 to avoid division by zero
    grouped['non_event_perc'] = grouped[0] / (grouped[0] + grouped[1] + 0.001)
    
    # Calculate WOE
    grouped['woe'] = np.log(grouped['non_event_perc'] / grouped['event_perc'] + 0.001)  # Add 0.001 to avoid log(0)
    
    return grouped['woe']

In [50]:
# Function to preprocess data using WOE

def preprocess_data(data):
    woe_data = data.copy()
    
    # Check if 'FraudResult' column exists
    if 'FraudResult' not in woe_data.columns:
        raise ValueError("Column 'FraudResult' not found in the data.")
    
    # Convert 'TransactionStartTime' to datetime
    woe_data['TransactionStartTime'] = pd.to_datetime(woe_data['TransactionStartTime'])
    
    # Calculate WOE for each feature
    for feature in data.columns:
        if feature == 'FraudResult':
            continue  # Skip the target variable
        if data[feature].dtype == 'object':
            woe_data[feature + '_woe'] = calculate_woe_categorical(data, feature, 'FraudResult')
        else:
            woe_data[feature + '_woe'] = calculate_woe_continuous(data, feature, 'FraudResult')
    
    return woe_data
data_preprocessed = preprocess_data(data)


AssertionError: 

In [38]:


# Perform WoE Binning
def perform_woe_binning(rfm):
    woe_transformer = WOE()
    rfm_woe = rfm.copy()
    woe_transformer.fit(rfm_woe, rfm_woe['Label'])
    rfm_woe = woe_transformer.transform(rfm_woe)
    return rfm_woe
rfm_woe = perform_woe_binning(rfm)

c:\Users\akili\Desktop\kifiya\week66\BatiBank_creditScoring\.venv\Lib\site-packages\xverse\transformer\_binning.py:159: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  bins_X_grouped = bins_X.groupby('Bins', as_index=True)
c:\Users\akili\Desktop\kifiya\week66\BatiBank_creditScoring\.venv\Lib\site-packages\xverse\transformer\_binning.py:159: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  bins_X_grouped = bins_X.groupby('Bins', as_index=True)
c:\Users\akili\Desktop\kifiya\week66\BatiBank_creditScoring\.venv\Lib\site-packages\xverse\transformer\_binning.py:159: FutureWarning: The default of observed=False is deprec

AttributeError: module 'pandas.core.algorithms' has no attribute 'quantile'